In [7]:
# Load Packages 
import pandas as pd 
import os
import glob
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
import numpy as np


In [14]:
# Indonesia Data including '[Units]' Data 
indonesia_full['[Units]'].value_counts()

PERCENT                   638
NUMBER                    354
PER_100000_POP             69
INDEX                      65
CU_USD                     50
KMSQ                       47
CON_USD                    33
NUM_TH                     30
Ratio                      29
PER_1000_LIVE_BIRTHS       24
CU_USD_M                   24
CON_USD_M                  20
HA_TH                      18
W_PER_CAPITA               16
CUR_LCU_M                   8
NUM_M                       8
SCORE                       8
PER_100_POP                 8
TEU                         8
PER_1000_POP                8
USD/m3                      7
TONNES_M                    7
KG_PER_CON_USD              7
MJ_PER_GDP_CON_PPP_USD      7
TONNES                      6
BOOL                        6
PER_POP_U5                  6
T_PER_HA                    6
mgr/m^3                     5
PER_1000000_POP             3
Name: [Units], dtype: int64

In [9]:
# Path for missing data directories - each indicator is saved as a seperate CSV 
path_g = '../../../Data/Data_preprocessed_for_PCA/Filtered_indicators_with_eligible_measurements_missing_data/Guatemala/'
path_i = '../../../Data/Data_preprocessed_for_PCA/Filtered_indicators_with_eligible_measurements_missing_data/Indonesia/'

extension = 'csv'

# generate path list for guatemala
guatemala_paths = glob.glob(path_g+'*.{}'.format(extension))

# generate csv list for guatemala 
guatemala_list = []
for string in guatemala_paths: 
    string = string.replace(path_g,'')
    guatemala_list.append(string)


# generate path list for indonesia 
indonesia_paths = glob.glob(path_i+'*.{}'.format(extension))

# generate csv list for indonesia 
indonesia_list = []
for string in indonesia_paths: 
    string = string.replace(path_i,'')
    indonesia_list.append(string)
indonesia_list[1:10]

['Indicator13.1.2.csv',
 'Indicator8.8.2.csv',
 'Indicator17.9.1.csv',
 'Indicator13.1.1.csv',
 'Indicator3.b.1.csv',
 'Indicator11.6.2.csv',
 'Indicator10.4.1.csv',
 'Indicator6.5.1.csv',
 'Indicator1.a.2.csv']

In [13]:
# Indonesia Imputation Code - Linear Regression 
# Read in full dataset with series code and units - this df includes non-disaggregated indicator measures' series codes and their units 
indonesia_full = pd.read_csv('https://raw.githubusercontent.com/PeishanLi/G5055_Practicum_Project2/main/Data/Indonesia%20Data%20Without%20Disaggregation.csv')
series_units = indonesia_full[['SeriesCode','[Units]']]
series_units = series_units.drop_duplicates()

# iteratively impute missing data - this for loop will include notes; guatemala follows the same structure (notes in this python chunk only)
path_i_out = '../../../Data/Data_preprocessed_for_PCA/Indicators_with_imputation/Indonesia/'
for csv in indonesia_list: 
    df = pd.read_csv(path_i+csv)
    for col in df.loc[:, df.columns != 'Year']:
        if (df[col].isnull().sum(axis=0) 0 == 0):
            df[str(col)+'_new'] = df[col]

        if (df[col].notnull().sum(axis=0) > 1) & (df[col].isnull().sum(axis=0) >= 1):
            missing_rows = df[df[col].isnull()]
            nonmissing_rows = df[df[col].notnull()]
            missing_years = missing_rows[['Year']]
            
            # Select non missing rows to train a linear regression model 
            y = nonmissing_rows[col]
            X = nonmissing_rows[['Year']]
            lm = LinearRegression().fit(X,y)
            
            # impute for missing rows based on trained model. 
            missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
            missing_rows["inferred_col"] = lm.predict(missing_years)
            # set lower bound of 0 
            missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
            
            # logical upper bounds baased on unit type - 'per n' sets max at n, percent sets max at 100, etc.
            for i in np.logspace(0,10,10):
                if (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='PER_'+str(i)+'_POP') is True: 
                    missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=i)
            if (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='PERCENT') is True:
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=100)
            elif (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='SCORE') is True: 
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=10)
            elif (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='Ratio') is True: 
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=1)
            
            # add this imputed data into a new column, and drop original column for this new df. 
            missing_rows = missing_rows[["Year","inferred_col"]]
            df = df.merge(missing_rows, on='Year', how ='left')
            df[str(col)+'_new'] = np.where(df[col].isnull(),df["inferred_col"],df[col])
            df = df.drop(columns = ["inferred_col", str(col)])
        
        # drop columns with 1 or fewer records 
        elif df[col].notnull().sum(axis=0) <= 1:
            df = df.drop(columns = [str(col)])
            
    df.to_csv(path_i_out + csv)

<ipython-input-13-beda5439873b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-13-beda5439873b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-13-beda5439873b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-13-beda5439873b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-13-beda5439873b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-13-beda5439873b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-13-beda5439873b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-13-beda5439873b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-13-beda5439873b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-13-beda5439873b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-13-beda5439873b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-13-beda5439873b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-13-beda5439873b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-13-beda5439873b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
<ipython-input-13-beda5439873b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

...

<ipython-input-13-beda5439873b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-13-beda5439873b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-13-beda5439873b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-13-beda5439873b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-13-beda5439873b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
<ipython-input-13-beda5439873b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

<ipython-input-13-beda5439873b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-13-beda5439873b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-13-beda5439873b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-13-beda5439873b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-13-beda5439873b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
<ipython-input-13-beda5439873b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

<ipython-input-13-beda5439873b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-13-beda5439873b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-13-beda5439873b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [12]:
# Guatemala Imputation Code - Linear Regression 

guatemala_full = pd.read_csv('https://raw.githubusercontent.com/PeishanLi/G5055_Practicum_Project2/main/Data/Guatmala%20Data%20Without%20Disaggregation.csv')
series_units = guatemala_full[['SeriesCode','[Units]']]
series_units = series_units.drop_duplicates()

path_g_out = '../../../Data/Data_preprocessed_for_PCA/Indicators_with_imputation/Guatemala/'
for csv in guatemala_list: 
    df = pd.read_csv(path_g+csv)
    for col in df.loc[:, df.columns != 'Year']:
        if (df[col].notnull().sum(axis=0) > 1) & (df[col].isnull().sum(axis=0) >= 1):
            missing_rows = df[df[col].isnull()]
            nonmissing_rows = df[df[col].notnull()]
            missing_years = missing_rows[['Year']]
            y = nonmissing_rows[col]
            X = nonmissing_rows[['Year']]
            lm = LinearRegression().fit(X,y)
            missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
            missing_rows["inferred_col"] = lm.predict(missing_years)
            missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
            for i in np.logspace(0,10,10):
                if (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='PER_'+str(i)+'_POP') is True: 
                    missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=i)
            if (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='PERCENT') is True:
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=100)
            elif (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='SCORE') is True: 
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=10)
            elif (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='Ratio') is True: 
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=1)
            missing_rows = missing_rows[["Year","inferred_col"]]
            df = df.merge(missing_rows, on='Year', how ='left')
            df[str(col)+'_new'] = np.where(df[col].isnull(),df["inferred_col"],df[col])
            df = df.drop(columns = ["inferred_col", str(col)])
        elif df[col].notnull().sum(axis=0) <= 1:
            df = df.drop(columns = [str(col)])
    df.to_csv(path_g_out + csv)
    

<ipython-input-12-527384756736>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-12-527384756736>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-12-527384756736>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-12-527384756736>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-12-527384756736>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
<ipython-input-12-527384756736>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

<ipython-input-12-527384756736>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-12-527384756736>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-12-527384756736>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-12-527384756736>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-12-527384756736>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-12-527384756736>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-12-527384756736>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-12-527384756736>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-12-527384756736>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

...

<ipython-input-12-527384756736>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-12-527384756736>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-12-527384756736>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-12-527384756736>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-12-527384756736>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-12-527384756736>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-12-527384756736>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-12-527384756736>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
<ipython-input-12-527384756736>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

<ipython-input-12-527384756736>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-12-527384756736>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-12-527384756736>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-12-527384756736>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
<ipython-input-12-527384756736>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows["inferred_col"] = lm.predict(missing_years)
<ipython-input-12-527384756736>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati